## TODO list

- Consider separating by gender or highly weighting gender. Gender is rarely wrong.
- Create weights for features. What's the best way to do this? I feel like there should be some objective function that we're maximizing/minimizing when creating the model.
- Tune hyperparameters
- Implement recall test. Using known links, check if the true match is in the nearest neighbors that we found.
- Think about the best way to impute the immigration year. Right now I'm just using if they are an immigrant or not.
- After finding the best way to tune weights, maybe we could skip linking algos entirely by just choosing the nearest neighbor. It's not terrible if this doesn't pan out, but it has huge upside if it does.

In [1]:
import pandas as pd
import numpy  as np
import nmslib
import time
from sklearn.model_selection import train_test_split
import pickle as pkl

In [2]:
with open("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/metric_transformer2.itml", "rb") as file:
    transformer = pkl.load(file)

In [3]:
#FIXME add fbp, mbp geo-coordinates to the test data.
data = pd.read_csv("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/florida_test_data.csv")

In [4]:
data.tail()

,index1910,marstat,birth_year,household1910,immigration,race,rel,female,bp,mbp,...,city,county,event_lat,event_lon,bplace_lat,bplace_lon,mbp_lat,mbp_lon,fbp_lat,fbp_lon
697460,8959876,2.0,1872.0,2150089.0,NaN,1,-1,0,46,70,...,Lakewood,Walton,30.988630,-86.281776,39.613350,-74.728561,42.999603,-107.551610,42.999603,-107.55161
697461,9199422,0.0,1907.0,2109664.0,NaN,0,2,0,22,22,...,NaN,Palm Beach,26.714390,-80.053192,28.566046,-81.688652,28.566046,-81.688652,42.999603,-107.55161
697462,8981019,1.0,1857.0,2085061.0,NaN,0,0,0,36,70,...,Inglis,Levy,29.030251,-82.668716,38.818199,-76.158730,42.999603,-107.551610,42.999603,-107.55161
697463,9199406,0.0,1905.0,2109664.0,NaN,0,3,1,22,22,...,NaN,Palm Beach,26.714390,-80.053192,28.566046,-81.688652,28.566046,-81.688652,42.999603,-107.55161
697464,9216227,1.0,1879.0,2121418.0,NaN,0,10,0,70,70,...,Bartow,Polk,27.896416,-81.843140,42.999603,-107.551610,42.999603,-107.551610,42.999603,-107.55161


In [5]:
cols_to_rename = {"int_place": "bp", "county_int": "county", "state_int": "state"}
cols_to_drop = ["household1910", "midinit", "cohort1", "cohort2", "bp", "mbp", "fbp", "city", "county", "state", "last_sdxn", "first_sdxn", "pr_name_gn", "pr_name_surn"]
data = (data.assign(immigrant=(~data.immigration.isnull()).astype(int))
            .assign(immigration=(data.immigration.mask(data.immigration.isnull(), 0)))
            .rename(cols_to_rename, axis=1)
            .drop(cols_to_drop, axis=1))
data = data.loc[(data.female != -1) & (data.race != -1) & (data.rel != -1) & (data.marstat != -1), :]
data = data.dropna()

In [6]:
dummies_list = ["marstat", "race", "rel","first_init", "last_init"]
data[dummies_list] = data[dummies_list].astype("category")
data = pd.concat([data.drop(dummies_list, axis=1), 
                  pd.get_dummies(data[dummies_list], prefix=dummies_list)], axis=1)

In [7]:
for i in data.columns:
    print(i)

index1910
birth_year
immigration
female
first_vec2
first_vec3
first_vec4
first_vec5
first_vec6
first_vec7
first_vec8
first_vec9
first_vec10
first_vec11
first_vec12
first_vec13
first_vec14
first_vec15
first_vec16
first_vec17
first_vec18
first_vec19
first_vec20
first_vec21
first_vec22
first_vec23
first_vec24
first_vec25
first_vec26
first_vec27
first_vec28
first_vec29
first_vec30
first_vec31
first_vec32
first_vec33
first_vec34
first_vec35
first_vec36
first_vec37
first_vec38
first_vec39
first_vec40
first_vec41
first_vec42
first_vec43
first_vec44
first_vec45
first_vec46
first_vec47
first_vec48
first_vec49
first_vec50
first_vec51
first_vec52
first_vec53
first_vec54
first_vec55
first_vec56
first_vec57
first_vec58
first_vec59
first_vec60
first_vec61
first_vec62
first_vec63
first_vec64
first_vec65
first_vec66
first_vec67
first_vec68
first_vec69
first_vec70
first_vec71
first_vec72
first_vec73
first_vec74
first_vec75
first_vec76
first_vec77
first_vec78
first_vec79
first_vec80
first_vec81
first_ve

In [8]:
if "rel_20" in data.columns:
    data = data.drop(["rel_20"], axis=1)
if "first_init_24" not in data.columns:
    data["first_init_24"] = 0
if "last_init_0" not in data.columns:
    data["last_init_0"] = 0
cols = ["index1910", "birth_year", "immigration", "female"]
cols.extend([f"first_vec{i}" for i in range(2, 202)])
cols.extend([f"last_vec{i}" for i in range(2, 202)])
cols.extend(["event_lat", "event_lon", "bplace_lat", "bplace_lon", "mbp_lat", "mbp_lon", "fbp_lat", "fbp_lon", "immigrant",
            "marstat_0.0", "marstat_1.0", "marstat_2.0", "marstat_3.0", "race_0", "race_1", "race_2"])
cols.extend([f"rel_{i}" for i in range(20)])
cols.extend([f"first_init_{i}" for i in range(1, 25)])
cols.extend(["first_init_25", "first_init_26"])
cols.extend([f"last_init_{i}" for i in range(0, 24)])
cols.extend(["last_init_25", "last_init_26"])
data = data[cols]
for i in data.columns:
    print(i)

index1910
birth_year
immigration
female
first_vec2
first_vec3
first_vec4
first_vec5
first_vec6
first_vec7
first_vec8
first_vec9
first_vec10
first_vec11
first_vec12
first_vec13
first_vec14
first_vec15
first_vec16
first_vec17
first_vec18
first_vec19
first_vec20
first_vec21
first_vec22
first_vec23
first_vec24
first_vec25
first_vec26
first_vec27
first_vec28
first_vec29
first_vec30
first_vec31
first_vec32
first_vec33
first_vec34
first_vec35
first_vec36
first_vec37
first_vec38
first_vec39
first_vec40
first_vec41
first_vec42
first_vec43
first_vec44
first_vec45
first_vec46
first_vec47
first_vec48
first_vec49
first_vec50
first_vec51
first_vec52
first_vec53
first_vec54
first_vec55
first_vec56
first_vec57
first_vec58
first_vec59
first_vec60
first_vec61
first_vec62
first_vec63
first_vec64
first_vec65
first_vec66
first_vec67
first_vec68
first_vec69
first_vec70
first_vec71
first_vec72
first_vec73
first_vec74
first_vec75
first_vec76
first_vec77
first_vec78
first_vec79
first_vec80
first_vec81
first_ve

In [9]:
data_matrix = data
data_ids = data_matrix.index1910.values
data_matrix = data_matrix.drop(["index1910"], axis=1).values
data_matrix = transformer.transform(data_matrix)

In [10]:
M = 30
efC = 500

num_threads = 8
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}

In [11]:
# Number of neighbors 
K=10

In [12]:
# Space name should correspond to the space name 
# used for brute-force search
space_name='l2'

In [14]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(data_matrix, ids=data_ids)

594699

In [15]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 30, 'indexThreadQty': 8, 'efConstruction': 500, 'post': 0}
Indexing time = 273.727360


In [16]:
# Setting query-time parameters
efS = 500
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 500}


In [17]:
data = pd.read_csv("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/florida1920_query_sample.csv", nrows=100)
cols_to_rename = {"int_place": "bp", "county_int": "county", "state_int": "state"}
cols_to_drop = ["household1920", "midinit", "cohort1", "cohort2", "bp", "mbp", "fbp", "county", "state", "last_sdxn", "first_sdxn"]

data = (data.assign(immigrant=(~data.immigration.isnull()).astype(int))
            .assign(immigration=(data.immigration.mask(data.immigration.isnull(), 0)))
            .rename(cols_to_rename, axis=1)
            .drop(cols_to_drop, axis=1))
data = data.loc[(data.female != -1) & (data.race != -1) & (data.rel != -1) & (data.marstat != -1), :]

dummies_list = ["marstat", "race", "rel", "first_init", "last_init"]
data[dummies_list] = data[dummies_list].astype("category")
data = pd.concat([data.drop(dummies_list, axis=1), 
                  pd.get_dummies(data[dummies_list], prefix=dummies_list)], axis=1)

In [18]:
not_in_cols = ["marstat_3.0", "race_2", "rel_5", "rel_6", "rel_9", "rel_10", "rel_11", "rel_12", "rel_13", 
               "rel_14", "rel_15", "rel_16", "rel_17", "rel_18", "rel_19", "first_init_2", 
               "first_init_9", "first_init_11", "first_init_17", "first_init_21", "first_init_24", "first_init_25", 
               "first_init_26", "last_init_0", "last_init_4", "last_init_6", "last_init_9", "last_init_17", 
               "last_init_21", "last_init_22", "last_init_25", "last_init_26"]
for i in not_in_cols:
    if i not in data.columns:
        data[i] = 0
if "rel_20" in data.columns:
    data = data.drop(["rel_20"], axis=1)
if "first_init_24" not in data.columns:
    data["first_init_24"] = 0
if "last_init_0" not in data.columns:
    data["last_init_0"] = 0
cols = ["index1920", "birth_year", "immigration", "female"]
cols.extend([f"first_vec{i}" for i in range(2, 202)])
cols.extend([f"last_vec{i}" for i in range(2, 202)])
cols.extend(["event_lat", "event_lon", "bplace_lat", "bplace_lon", "mbp_lat", "mbp_lon", "fbp_lat", "fbp_lon", "immigrant",
            "marstat_0.0", "marstat_1.0", "marstat_2.0", "marstat_3.0", "race_0", "race_1", "race_2"])
cols.extend([f"rel_{i}" for i in range(20)])
cols.extend([f"first_init_{i}" for i in range(1, 27)])
cols.extend([f"last_init_{i}" for i in range(0, 24)])
cols.extend(["last_init_25", "last_init_26"])
data = data[cols]
for i in data.columns:
    print(i)

index1920
birth_year
immigration
female
first_vec2
first_vec3
first_vec4
first_vec5
first_vec6
first_vec7
first_vec8
first_vec9
first_vec10
first_vec11
first_vec12
first_vec13
first_vec14
first_vec15
first_vec16
first_vec17
first_vec18
first_vec19
first_vec20
first_vec21
first_vec22
first_vec23
first_vec24
first_vec25
first_vec26
first_vec27
first_vec28
first_vec29
first_vec30
first_vec31
first_vec32
first_vec33
first_vec34
first_vec35
first_vec36
first_vec37
first_vec38
first_vec39
first_vec40
first_vec41
first_vec42
first_vec43
first_vec44
first_vec45
first_vec46
first_vec47
first_vec48
first_vec49
first_vec50
first_vec51
first_vec52
first_vec53
first_vec54
first_vec55
first_vec56
first_vec57
first_vec58
first_vec59
first_vec60
first_vec61
first_vec62
first_vec63
first_vec64
first_vec65
first_vec66
first_vec67
first_vec68
first_vec69
first_vec70
first_vec71
first_vec72
first_vec73
first_vec74
first_vec75
first_vec76
first_vec77
first_vec78
first_vec79
first_vec80
first_vec81
first_ve

In [19]:
data = data.dropna()
query_ids = data.index1920.values
query_matrix = data.drop(["index1920"], axis=1).values

In [20]:
query_matrix.shape

(84, 491)

In [21]:
query_matrix = transformer.transform(query_matrix)

In [22]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

kNN time total=0.050894 (sec), per query=0.000606 (sec), per query adjusted for thread number=0.004847 (sec)


In [23]:
nbrs

[(array([9182677, 9336183, 9198369, 9362899, 9293265, 9377610, 9281999,
         9165065, 9283449, 9120163], dtype=int32),
  array([ 7.1997085,  9.1180105,  9.124964 ,  9.564973 ,  9.574142 ,
          9.771056 ,  9.962169 ,  9.991009 , 10.097485 , 10.196792 ],
        dtype=float32)),
 (array([9530512, 9150928, 9237217, 9502247, 9545113, 9594169, 9125771,
         9027362, 9240936, 9577112], dtype=int32),
  array([5.996275 , 7.105719 , 7.1291156, 7.364417 , 7.416989 , 7.4380236,
         7.5276685, 7.545123 , 7.5735297, 7.6236625], dtype=float32)),
 (array([9445044, 9614656, 9304893, 9572499, 9126271, 8983186, 9422717,
         9368262, 8983086, 9127079], dtype=int32),
  array([3.4117537, 3.5536556, 3.7083757, 3.7543354, 4.1966944, 4.209249 ,
         4.2338057, 4.687364 , 4.713523 , 4.8155417], dtype=float32)),
 (array([9193468, 9415097, 9254570, 8886594, 9325420, 9510712, 9182077,
         8883668, 9313261, 8962024], dtype=int32),
  array([3.426174 , 3.4558988, 3.6501842, 3.93846  ,

In [26]:
query_ids[19]

10872404

In [27]:
nbrs[19]

(array([9430328, 9035523, 9155419, 8962951, 8934995, 8900313, 9392300,
        9411226, 9311600, 9023221], dtype=int32),
 array([1.2561836, 2.1233768, 2.2359066, 2.5141325, 2.615968 , 2.8893704,
        3.0283093, 3.058474 , 3.2012172, 3.2832615], dtype=float32))

In [34]:
query_ids[28]

10879940

In [33]:
nbrs[28]

(array([8918333, 9080186, 9585162, 9117055, 8889112, 9109586, 9011607,
        9600336, 9541263, 9066303], dtype=int32),
 array([1.3818666, 2.6970868, 3.1507342, 3.3481987, 3.801887 , 3.860544 ,
        4.00905  , 4.0879316, 4.593873 , 4.64498  ], dtype=float32))